## Imports

In [1]:
import pandas as pd

In [2]:
dataframe = pd.read_csv('./Data/RepublicaGuatemala.csv')

In [3]:
dataframe

,CODIGO,DISTRITO,DEPARTAMENTO,MUNICIPIO,ESTABLECIMIENTO,DIRECCION,TELEFONO,SUPERVISOR,DIRECTOR,NIVEL,SECTOR,AREA,STATUS,MODALIDAD,JORNADA,PLAN,DEPARTAMENTAL
0,16-01-0138-46,16-031,ALTA VERAPAZ,COBAN,COLEGIO COBAN,KM.2 SALIDA A SAN JUAN CHAMELCO ZONA 8,77945104,PATRICIO NAJARRO ASENCIO,GUSTAVO ADOLFO SIERRA POP,DIVERSIFICADO,PRIVADO,URBANA,ABIERTA,MONOLINGUE,MATUTINA,DIARIO(REGULAR),ALTA VERAPAZ
1,16-01-0139-46,16-031,ALTA VERAPAZ,COBAN,COLEGIO PARTICULAR MIXTO VERAPAZ,KM 209.5 ENTRADA A LA CIUDAD,77367402,PATRICIO NAJARRO ASENCIO,GILMA DOLORES GUAY PAZ DE LEAL,DIVERSIFICADO,PRIVADO,URBANA,ABIERTA,MONOLINGUE,MATUTINA,DIARIO(REGULAR),ALTA VERAPAZ
2,16-01-0140-46,16-031,ALTA VERAPAZ,COBAN,"COLEGIO ""LA INMACULADA""",7A. AVENIDA 11-109 ZONA 6,78232301,PATRICIO NAJARRO ASENCIO,VIRGINIA SOLANO SERRANO,DIVERSIFICADO,PRIVADO,URBANA,ABIERTA,MONOLINGUE,MATUTINA,DIARIO(REGULAR),ALTA VERAPAZ
3,16-01-0141-46,16-005,ALTA VERAPAZ,COBAN,ESCUELA NACIONAL DE CIENCIAS COMERCIALES,2A CALLE 11-10 ZONA 2,79514215,NORA LILIANA FIGUEROA HERNÁNDEZ,HÉCTOR ROLANDO CHUN POOU,DIVERSIFICADO,OFICIAL,URBANA,ABIERTA,MONOLINGUE,MATUTINA,DIARIO(REGULAR),ALTA VERAPAZ
4,16-01-0142-46,16-005,ALTA VERAPAZ,COBAN,INSTITUTO NORMAL MIXTO DEL NORTE 'EMILIO ROSAL...,3A AVE 6-23 ZONA 11,79521468,NORA LILIANA FIGUEROA HERNÁNDEZ,VICTOR HUGO DOMÍNGUEZ REYES,DIVERSIFICADO,OFICIAL,URBANA,ABIERTA,BILINGUE,VESPERTINA,DIARIO(REGULAR),ALTA VERAPAZ
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6602,19-09-0040-46,19-021,ZACAPA,LA UNION,"LICEO PARTICULAR MIXTO ""JIREH""",BARRIO NUEVO,79418369,ASBEL IVÁN SÚCHITE ARROYO,ANA MARÍA CUELLAR GUERRA,DIVERSIFICADO,PRIVADO,URBANA,ABIERTA,MONOLINGUE,MATUTINA,DIARIO(REGULAR),ZACAPA
6603,19-09-0048-46,19-021,ZACAPA,LA UNION,"LICEO PARTICULAR MIXTO ""JIREH""",BARRIO NUEVO,79418369,ASBEL IVÁN SÚCHITE ARROYO,ANA MARÍA CUELLAR GUERRA,DIVERSIFICADO,PRIVADO,URBANA,ABIERTA,MONOLINGUE,SIN JORNADA,SEMIPRESENCIAL (UN DÍA A LA SEMANA),ZACAPA
6604,19-10-0013-46,19-015,ZACAPA,HUITE,INSTITUTO DIVERSIFICADO,BARRIO BUENOS AIRES,48579171,SILDY MARIELA PEREZ FRANCO,WUENDY JHOJANA SIERRA PAZ,DIVERSIFICADO,OFICIAL,URBANA,ABIERTA,MONOLINGUE,NOCTURNA,DIARIO(REGULAR),ZACAPA
6605,19-10-1009-46,19-015,ZACAPA,HUITE,INSTITUTO DIVERSIFICADO POR COOPERATIVA,BARRIO EL CAMPO,55958103,SILDY MARIELA PEREZ FRANCO,ROBIDIO PORTILLO SALGUERO,DIVERSIFICADO,COOPERATIVA,URBANA,ABIERTA,MONOLINGUE,VESPERTINA,DIARIO(REGULAR),ZACAPA


# CODIGO

# DISTRITO

# DEPARTAMENTO

# MUNICIPIO

# ESTABLECIMIENTO

# DIRECCION

# TELEFONO

# SUPERVISOR

# DIRECTOR

# NIVEL

# SECTOR

# AREA

# STATUS

# MODALIDAD

# JORNADA

# PLAN

# DEPARTAMENTAL